In [1]:
!pip install tensorflow keras opencv-python numpy scikit-learn pillow rembg django djangorestframework


In [5]:
!pip uninstall tensorflow -y


Found existing installation: tensorflow 2.16.2
Uninstalling tensorflow-2.16.2:
  Successfully uninstalled tensorflow-2.16.2


In [6]:
!pip install tensorflow-cpu  # Optimized for CPU


  Using cached tensorflow_cpu-2.16.2-cp312-cp312-macosx_10_15_x86_64.whl.metadata (4.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 MB 944.8 kB/s eta 0:00:0000:010:07m


In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from rembg import remove
from PIL import Image
import matplotlib.pyplot as plt

In [11]:
import os
import numpy as np
from PIL import Image
from rembg import remove
from sklearn.model_selection import train_test_split

# Define data directory and image size
data_dir = '/users/gorantlasaitejaswini/Desktop/mlplastic'
img_size = (224, 224)

# Get list of class names, ignoring non-directory files
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

X, y, weights = [], [], []

for class_idx, class_name in enumerate(classes):
    class_path = os.path.join(data_dir, class_name)
    
    # Ensure it's a directory before proceeding
    if not os.path.isdir(class_path):
        continue
    
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        
        # Skip hidden/system files like .DS_Store
        if img_name.startswith('.'):
            continue
        
        try:
            # Load image
            img = Image.open(img_path).convert('RGBA')
            img_no_bg = remove(img)  # Remove background
            img_no_bg = img_no_bg.convert('RGB')
            img_no_bg = img_no_bg.resize(img_size)
            img_array = np.array(img_no_bg) / 255.0  # Normalize
            
            # Extract weight from filename (assuming weight is in filename like 'plastic_100g.jpg')
            weight = int(''.join(filter(str.isdigit, img_name)))
            
            X.append(img_array)
            y.append(class_idx)
            weights.append(weight)
        
        except Exception as e:
            print(f"Skipping {img_name}: {e}")  # Print error but continue processing

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)
weights = np.array(weights)

# Train-Test Split
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")


Training samples: 176, Test samples: 44


In [21]:
inputs = keras.Input(shape=(224, 224, 3))
x = keras.layers.Conv2D(32, (3,3), activation='relu')(inputs)
x = keras.layers.MaxPooling2D((2,2))(x)
x = keras.layers.Conv2D(64, (3,3), activation='relu')(x)
x = keras.layers.MaxPooling2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)

# Output layers
classification_output = keras.layers.Dense(len(classes), activation='softmax', name='class_output')(x)
weight_output = keras.layers.Dense(1, activation='linear', name='weight_output')(x)

# Final Model
model = keras.Model(inputs=inputs, outputs=[classification_output, weight_output])
model.compile(optimizer='adam',
              loss=['sparse_categorical_crossentropy', 'mse'],  # Loss for each output
              metrics=[['accuracy'], ['mae']])  # Metric for each output


In [22]:
history = model.fit(X_train, [y_train, weights_train], epochs=10, validation_data=(X_test, [y_test, weights_test]))


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 801ms/step - class_output_accuracy: 0.1823 - class_output_loss: 3.1563 - loss: 635.7602 - weight_output_loss: 629.6469 - weight_output_mae: 20.5220 - val_class_output_accuracy: 0.1364 - val_class_output_loss: 2.1240 - val_loss: 404.2490 - val_weight_output_loss: 494.6513 - val_weight_output_mae: 16.3031
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 720ms/step - class_output_accuracy: 0.1692 - class_output_loss: 2.6537 - loss: 356.7786 - weight_output_loss: 352.5097 - weight_output_mae: 15.2688 - val_class_output_accuracy: 0.1364 - val_class_output_loss: 4.6588 - val_loss: 230.6792 - val_weight_output_loss: 276.4490 - val_weight_output_mae: 12.2664
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 716ms/step - class_output_accuracy: 0.1409 - class_output_loss: 5.4522 - loss: 180.9998 - weight_output_loss: 175.8491 - weight_output_mae: 10.4455 - val_class_output_accuracy: 0.1591 - val_class_output_loss: 7.4397 - val_loss: 218.5619 - val_weight_output_loss: 205.3380 -

In [23]:
model.save('plastic_model.keras') 

In [7]:
import numpy as np
from PIL import Image
from rembg import remove
import keras

# Load trained model
model = keras.models.load_model('plastic_model.keras')

# Define expected image size and class labels (Ensure order matches model training)
img_size = (224, 224)  
classes = ['hdpe', 'ldpe', 'others', 'pet', 'pp', 'ps', 'pvc']  

def predict_plastic(img_path):
    try:
        print("Opening image...")
        img = Image.open(img_path).convert('RGBA')

        print("Removing background...")
        img_no_bg = remove(img).convert('RGB')

        print("Resizing image...")
        img_no_bg = img_no_bg.resize(img_size)  

        print("Converting to NumPy array...")
        img_array = np.array(img_no_bg, dtype=np.float32) / 255.0  

        print("Expanding dimensions for model input...")
        img_array = np.expand_dims(img_array, axis=0)  

        print("Image shape after processing:", img_array.shape)

        # Debugging: Check pixel values
        print("First pixel values:", img_array[0, 0, 0, :])

        print("Making prediction...")
        outputs = model.predict(img_array)
        print("Raw model output:", outputs)

        if isinstance(outputs, list) and len(outputs) == 2:
            class_pred, weight_pred = outputs
        else:
            print("Unexpected output shape from model:", outputs)
            return None, None

        # Debugging: Check class probabilities
        print("Class Probabilities:", class_pred)
        print("Predicted Class Index:", np.argmax(class_pred))

        predicted_class = classes[np.argmax(class_pred)]
        predicted_weight = float(weight_pred[0][0])  

        return predicted_class, predicted_weight
    
    except Exception as e:
        print("Error during prediction:", str(e))
        return None, None

# Test prediction
img_path = '/users/gorantlasaitejaswini/Desktop/test/00000019.jpg'
pred_class, pred_weight = predict_plastic(img_path)
if pred_class is not None:
    print(f"Predicted Class: {pred_class}, Predicted Weight: {pred_weight}g")


Opening image...
Removing background...
Resizing image...
Converting to NumPy array...
Expanding dimensions for model input...
Image shape after processing: (1, 224, 224, 3)
First pixel values: [0. 0. 0.]
Making prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Raw model output: [array([[2.4626857e-02, 1.1497013e-03, 3.5142279e-05, 6.4787902e-02,
        1.9102610e-03, 8.9812422e-01, 9.3658781e-03]], dtype=float32), array([[2.1138558]], dtype=float32)]
Class Probabilities: [[2.4626857e-02 1.1497013e-03 3.5142279e-05 6.4787902e-02 1.9102610e-03
  8.9812422e-01 9.3658781e-03]]
Predicted Class Index: 5
Predicted Class: ps, Predicted Weight: 2.1138558387756348g


In [27]:
import os
print(os.path.exists(img_path))  # Should print True if the file exists


True


In [28]:
from rembg import remove

img = Image.open(img_path).convert('RGBA')
img_no_bg = remove(img).convert('RGB')
img_no_bg.show()  # Show the image to confirm background removal


In [29]:
print("Image shape before expansion:", img_no_bg.size)
print("Image array shape after processing:", img_array.shape)


Image shape before expansion: (224, 224)
Image array shape after processing: (224, 224, 3)
